# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
vacation_data=pd.read_csv("../output_data/cities.csv")
vacation_df=pd.DataFrame(vacation_data)
vacation_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Lavrentiya,0,RU,1597115437,70,65.58,-171.00,53.37,10.07
1,Serik,4,TR,1597115438,73,36.92,31.10,75.20,4.70
2,Punta Arenas,100,CL,1597115284,89,-53.15,-70.92,32.00,9.17
3,Husavik,100,IS,1597115441,67,66.04,-17.34,59.00,10.29
4,El Dorado,1,US,1597115151,94,33.21,-92.67,78.80,3.83
...,...,...,...,...,...,...,...,...,...
592,Gusinoye Ozero,0,RU,1597116112,48,51.11,106.26,70.20,4.45
593,Ampanihy,15,MG,1597116113,93,-24.70,44.75,57.67,2.71
594,Blackwater,0,AU,1597116114,37,-23.58,148.88,68.68,2.35
595,Santa Maria,100,BR,1597116115,37,-29.68,-53.81,82.40,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
gmaps.configure(api_key = g_key)

locations = vacation_df[["Latitude", "Longitude"]]
humidity = vacation_df["Humidity"].astype(float)
max_humidity = humidity.max()

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=5)

fig.add_layer(heatmap_layer)

plt.savefig("../Images/heatmap.png")

fig



Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
city_weather=vacation_df.loc[(vacation_df["Max Temp"]>70) & (vacation_df["Max Temp"]<80) & 
                             (vacation_df["Wind Speed"]<10) & (vacation_df["Cloudiness"]==0)].dropna()
city_weather

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
157,Sancti Spíritus,0,CU,1597115617,89,21.93,-79.44,74.82,7.02
194,Alvarães,0,BR,1597115658,77,-3.22,-64.80,75.92,2.13
242,Pontes e Lacerda,0,BR,1597115713,43,-15.23,-59.34,72.75,1.61
243,Kumul,0,CN,1597115714,46,42.80,93.45,76.41,9.40
254,Tigil',0,RU,1597115727,47,57.80,158.67,71.17,7.00
270,Corinth,0,GR,1597115746,48,37.94,22.96,71.01,1.01
272,Khandyga,0,RU,1597115748,43,62.67,135.60,71.78,3.56
274,Yeppoon,0,AU,1597115750,35,-23.13,150.73,71.60,3.36
296,Turukhansk,0,RU,1597115774,54,65.82,87.98,70.75,2.73
299,Morros,0,BR,1597115778,51,-9.45,-46.30,70.20,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = city_weather.loc[:,["City","Country", "Latitude", "Longitude"]]


hotel_df["Hotel Name"] = ""


hotel_df

,City,Country,Latitude,Longitude,Hotel Name
157,Sancti Spíritus,CU,21.93,-79.44,
194,Alvarães,BR,-3.22,-64.80,
242,Pontes e Lacerda,BR,-15.23,-59.34,
243,Kumul,CN,42.80,93.45,
254,Tigil',RU,57.80,158.67,
270,Corinth,GR,37.94,22.96,
272,Khandyga,RU,62.67,135.60,
274,Yeppoon,AU,-23.13,150.73,
296,Turukhansk,RU,65.82,87.98,
299,Morros,BR,-9.45,-46.30,


In [39]:
# constructing the gmaps inqury with parameters 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "hotel",
    "key": g_key,
}


In [ ]:
for index, row in hotel_df.iterrows():
    # getting city name, latitude, longitude from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    # adding keyword to params dictionary
    params["location"] = f"{lat},{lng}"
    # assembling the url and making API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    # gathering the results
    results = response['results']
    # saving the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/skipping.") 
    print("------------")
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1.01)
# Print end of search once searching is completed
print("-------Ending of Search-------")

Retrieving Results for Index 157: Sancti Spíritus.
Closest hotel in Sancti Spíritus is Sancti Spiritus.
------------
Retrieving Results for Index 194: Alvarães.
Closest hotel in Alvarães is Alvarães.
------------
Retrieving Results for Index 242: Pontes e Lacerda.
Closest hotel in Pontes e Lacerda is Centro.
------------
Retrieving Results for Index 243: Kumul.
Closest hotel in Kumul is Hami.
------------
Retrieving Results for Index 254: Tigil'.
Closest hotel in Tigil' is Tigil'.
------------
Retrieving Results for Index 270: Corinth.
Closest hotel in Corinth is Corinth.
------------
Retrieving Results for Index 272: Khandyga.
Closest hotel in Khandyga is Khandyga.
------------
Retrieving Results for Index 274: Yeppoon.
Closest hotel in Yeppoon is Yeppoon.
------------
Retrieving Results for Index 296: Turukhansk.
Closest hotel in Turukhansk is Turukhansk.
------------
Retrieving Results for Index 299: Morros.
Closest hotel in Morros is Morros.
------------
Retrieving Results for Inde

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [32]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

plt.savefig("../Images/hotelmap.png")

# Display Map
fig



Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>